In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class Data_Validation_Config:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [4]:
from src.DiamondPricePrediction.constants import *
from src.DiamondPricePrediction.utils.common import read_yaml,create_directories
import pandas as pd


In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self)->Data_Validation_Config:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])
        data_validation_config = Data_Validation_Config(
        root_dir= config.root_dir,
        STATUS_FILE= config.STATUS_FILE,
        unzip_data_dir= config.unzip_data_dir,
        all_schema=schema
        )
        return data_validation_config

In [6]:
import os 
from src.DiamondPricePrediction import logger
class Data_Validation:
    def __init__(self,config: Data_Validation_Config):
        self.config = config
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")
                        f.write(f"current col:{col}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e      


In [7]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = Data_Validation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-11-03 23:52:09,953: INFO: common:yaml file:config\config.yaml loaded successfully]
[2023-11-03 23:52:09,954: INFO: common:yaml file:params.yaml loaded successfully]
[2023-11-03 23:52:09,956: INFO: common:yaml file:schema.yaml loaded successfully]
[2023-11-03 23:52:09,957: INFO: common:created directory at: artifacts]
[2023-11-03 23:52:09,957: INFO: common:created directory at: artifacts/data_validation]
